In [1]:
optimizer: str = 'rmsprop'
epochs: int = 15
batch_size: int = 32

In [2]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential

# Load the IMDB dataset
max_features = 10000  # Number of words to consider as features
maxlen = 500  # Cuts off texts after this number of words (among the max_features most common words)

print("Loading data...")
(input_train, y_train), (input_test, y_test) = imdb.load_data(num_words=max_features)
print(len(input_train), "train sequences")
print(len(input_test), "test sequences")

# Pad sequences to a fixed length
print("Pad sequences (samples x time)")
input_train = sequence.pad_sequences(input_train, maxlen=maxlen)
input_test = sequence.pad_sequences(input_test, maxlen=maxlen)
print("input_train shape:", input_train.shape)
print("input_test shape:", input_test.shape)

# Size of the dataset
size = len(input_train) + len(input_test)

/Users/jules/Projects/notebooks/env/lib/python3.9/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


Loading data...
25000 train sequences
25000 test sequences
Pad sequences (samples x time)
input_train shape: (25000, 500)
input_test shape: (25000, 500)


In [3]:
# Build the RNN model
model = Sequential()
model.add(layers.Embedding(max_features, 32))
model.add(layers.SimpleRNN(32))
model.add(layers.Dense(1, activation='sigmoid'))

In [4]:
# Compile the model
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

In [5]:
from codecarbon import EmissionsTracker
from examples.serializer import Serializer

# Serialize the model
serializer = Serializer()
row = serializer.serialize(model, epochs, batch_size, size)

# Start the emissions tracker
tracker = EmissionsTracker(log_level='critical')
tracker.start()

# Train the model
print("Training model...")
model.fit(input_train, y_train, epochs=epochs, batch_size=batch_size, validation_split=0.2)

# Save the model carbon emissions
row['co2'] = tracker.stop()

Training model...
Epoch 1/15
625/625 [==============================] - 21s 33ms/step - loss: 0.5907 - accuracy: 0.6704 - val_loss: 0.4086 - val_accuracy: 0.8248
Epoch 2/15
625/625 [==============================] - 21s 33ms/step - loss: 0.3699 - accuracy: 0.8450 - val_loss: 0.3515 - val_accuracy: 0.8582
Epoch 3/15
625/625 [==============================] - 21s 34ms/step - loss: 0.2772 - accuracy: 0.8913 - val_loss: 0.3721 - val_accuracy: 0.8524
Epoch 4/15
625/625 [==============================] - 21s 34ms/step - loss: 0.2403 - accuracy: 0.9078 - val_loss: 0.3746 - val_accuracy: 0.8510
Epoch 5/15
625/625 [==============================] - 22s 34ms/step - loss: 0.2005 - accuracy: 0.9250 - val_loss: 0.4144 - val_accuracy: 0.8600
Epoch 6/15
625/625 [==============================] - 22s 34ms/step - loss: 0.1486 - accuracy: 0.9456 - val_loss: 0.4977 - val_accuracy: 0.7970
Epoch 7/15
625/625 [==============================] - 22s 35ms/step - loss: 0.1204 - accuracy: 0.9566 - val_loss: 0.45

In [6]:
# Evaluate the model on the test set
print("Evaluating model...")
accuracy = model.evaluate(input_test, y_test)[1]
print(f'Test accuracy: {accuracy}')

Evaluating model...
782/782 [==============================] - 6s 8ms/step - loss: 1.0662 - accuracy: 0.7508
Test accuracy: 0.7508400082588196


In [7]:
from csv import DictWriter

# Save the model metadata
with open('results.csv', 'a') as f:
    writer = DictWriter(f, fieldnames=row.keys())
    writer.writerow(row)